# Optimizer(옵티마이저)
모델의 오류를 줄이기 위해 모델 매개변수를 조정하는 과정.  
1. 옵티마이저 등록  
`optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate`  
2.변화도 명시적으로 0 설정  
`optimizer.zero_grad()`  
3. 예측 손실 역전파  
`loss.bachwards()`  
4. 역전파 단계에서 수집된 변화도로 매개변수 조정   
`optimizer.step()`  
매개변수 = 매개변수 - 학습률 * 그래디언트

In [2]:
# 기본 라이브러리 설치
import torch
import torch.nn as nn
import torch.optim as optim

# Learning Rate Scheduler  
learning rate: gradient 보폭  
Scheduler가 이것을 변경시켜주기도  
`optimizer=torch.optim.SGD(model.parameters(),lr=0.01)` 기본적으로 optimizer 넣어주고

In [ ]:
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

##LambdaLR

`scheduler=optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=lambda epoch:0.95**epoch)`